# Calculation of isoprenol concentrations for different strain designs

# Instructions

### 1. Clone the git repository with the `OMG` library 

`git clone -b omgforallhosts --single-branch https://github.com/JBEI/OMG.git`

or pull the latest version.

### 2. Copy the file containing mapping for CIDs

<!-- <font color='red'>
    
We need access to `mapping` directory from `OMG`.
    
Should we inlcude this already into `multiomics` repo?

</font> -->

In [1]:
%cp -r ../OMG/mapping .

____________________________

Update the OMG library from github

In [7]:
%cd ../OMG
!git pull 
%cd ../multiomicspaper/

/scratch/tradivojevic/OMG
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/JBEI/OMG
   e6a3686..7fe04fc  omgforallhosts -> origin/omgforallhosts
Updating e6a3686..7fe04fc
Fast-forward
 OMG_notebook.ipynb | 149 +++++++++++++++++++++++++++++------------------------
 omg.py             |  57 ++++++++++----------
 2 files changed, 111 insertions(+), 95 deletions(-)
/scratch/tradivojevic/multiomicspaper


-------------------

### 3. If running the `generate_omics_data.ipynb` notebook on the `jprime` juputerhub make sure you are using the `biodesign_3.7` kernel. Otherwise, follow the installation requirements for `OMG` library.

### 4. The two files that are necessary for the notebook to run are:
   - A modified E. coli model with the isopentenol pathway added to it (`iJO1366_MVA.json` file <font color='red'>in the `/models` directory</font>)
   - A set of designs (e.g. `data/training_data_8reactions.csv`) containing the details of which reactions are either:
       - (0) eliminated 
       - (1) included
       - (2) doubled the flux
   
   
### 5. Running the cells below <font color='red'>will generate this and that output</font>

# Imports

In [1]:
import sys
sys.path.append('../')

import OMG.omg as omg
import os

# User parameters passed to the program

In [3]:
user_params = {
    'host': 'ecoli', # ecoli or ropacus
    'modelfile': 'iJO1366_MVA.json',
    'modelfilepath': 'data/models/',
    'timestart': 0.0,
    'timestop': 8.0,
    'numpoints': 9,
    'designsfile': 'training_data_8reactions.csv',
    'designsfilepath': 'data_mo/',
    'numreactions': 8,
    'numinstances': 96
}

<font color='red'>
    
Changed 'trainingfile' to 'designsfile'.
    
Inlcude the path to directory with output files as user input.
</font>

# Set the global variables in the OMG code from the user parameters

In [4]:
omg.HOST_NAME = user_params['host']
omg.MODEL_FILENAME = user_params['modelfile']
omg.MODEL_FILEPATH = user_params['modelfilepath']
omg.TIMESTART = user_params['timestart']
omg.TIMESTOP = user_params['timestop']
omg.NUMPOINTS = user_params['numpoints']
omg.TRAINING_FILE_NAME = user_params['designsfile']
omg.TRAINING_FILE_PATH = user_params['designsfilepath']
omg.NUM_REACTIONS = user_params['numreactions']
omg.NUM_INSTANCES = user_params['numinstances']

<font color='red'>
Change 'TRAINING_FILE_NAME' to 'DESIGNS_FILE_NAME'.
</font>

# Run OMG code to generate synthetic data

<font color='red'>        
Why called `DATA_FILE_PATH` if it's for the model? This should be more clear.
</font>

In [5]:
%%time
# creating the file path to the model file
filename = os.path.join(omg.MODEL_FILEPATH, omg.MODEL_FILENAME)
omg.generate_data_for_host(filename)

Displaying the reaction bounds after constraining them:
(-10.0, 1000.0)
Displaying the reaction bounds after constraining them:
(-1000.0, 1000.0)
Displaying the reaction bounds after constraining them:
(-200.0, 1000.0)
Displaying the reaction bounds after constraining them:
(-1000.0, 1000.0)
Displaying the reaction bounds after constraining them:
(-1000.0, 1000.0)
Displaying the reaction bounds after constraining them:
(-1000.0, 1000.0)
Displaying the reaction bounds after constraining them:
(-1000.0, 1000.0)
Displaying the reaction bounds after constraining them:
(-200.0, 1000.0)
Displaying the reaction bounds after constraining them:
(-10.0, 1000.0)
Displaying the reaction bounds after constraining them:
(-839.5219917958988, 1000.0)
Displaying the reaction bounds after constraining them:
(-76.73388401633038, 1000.0)
Displaying the reaction bounds after constraining them:
(-1000.0, 1000.0)
Displaying the reaction bounds after constraining them:
(-710.6633672620774, 1000.0)
Displaying 

../OMG/omg.py:254 RuntimeWarning: invalid value encountered in double_scalars


Displaying the reaction bounds after constraining them:
(-14.476803483353784, 1000.0)
Displaying the reaction bounds after constraining them:
(-35.95174307885639, 1000.0)
Displaying the reaction bounds after constraining them:
(-0.8120280420812237, 1000.0)
Displaying the reaction bounds after constraining them:
(-0.540124539348402, 1000.0)
Displaying the reaction bounds after constraining them:
(-11.435539261593966, 1000.0)
Displaying the reaction bounds after constraining them:
(-1.0542687781548568, 1000.0)
Displaying the reaction bounds after constraining them:
(-55.111244447101434, 1000.0)
Displaying the reaction bounds after constraining them:
(-0.0, 1000.0)
Displaying the reaction bounds after constraining them:
(-14.476803483353784, 1000.0)
Displaying the reaction bounds after constraining them:
(-35.95174307885639, 1000.0)
Displaying the reaction bounds after constraining them:
(-0.8120280420812237, 1000.0)
Displaying the reaction bounds after constraining them:
(-0.540124539348

cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


-------------------------------------
Isopentenol concentrations:
 0.0    0.000000
1.0    0.003354
2.0    0.012094
3.0    0.034873
4.0    0.094241
5.0    0.248968
6.0    0.774787
7.0    0.774787
8.0    0.774787
9.0    0.774787
dtype: float64
String index: 95
Flux value for isopentenol from MOMA calculation: 90.30121886839387
Isopentenol concentration for this strain: 0.7747865691137857
-------------------------------------
CPU times: user 1h 28min 52s, sys: 38.6 s, total: 1h 29min 30s
Wall time: 1h 29min 42s
